# ```esgRating.ipynb```
- 데이터를 수집, 전처리하여 ```esgRating.csv```를 만든 파일

## ESG Rating
- 등급 산출 시기 관련
  - 2023년 1월 말에 2023년 ESG등급을 확인할 수 있으며,
  - 2022년의 경우 2022년 11월에 조정된 것을 확인할 수 있었다.
    ![2023-02-01, 한국ESG기준원 등급 조회, 비고.png](../fig/md/2023-02-01%2C%20%ED%95%9C%EA%B5%ADESG%EA%B8%B0%EC%A4%80%EC%9B%90%20%EB%93%B1%EA%B8%89%20%EC%A1%B0%ED%9A%8C%2C%20%EB%B9%84%EA%B3%A0.png)

# import

In [6]:
import os
import sys
import time
import pickle
import warnings
import urllib.request
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import requests
from bs4 import BeautifulSoup as bs

import FinanceDataReader as fdr
from tqdm import tqdm


warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
# pd.options.display.float_format = '{:.4f}'.format
plt.style.use("ggplot")
%config InlineBackend.figure_format = 'retina'


sys.path.append("../import")
import module as m

data_path = m.data_path
fp_esg = f"""{m.fp["esgRating"]}"""


fp_esg_1118 = f"{data_path}esg_1118.parquet"
fp_esg_21 = f"{data_path}esg_21.parquet"

# ```KCGS_ESGRating(2011~2018).csv```
  - 개요
    - 2011~2018년 ESG등급
  - Data Collection Method
    - Web Scraping
  - Data Source
    - [KRX 정보데이터시스템](https://data.krx.co.kr/contents/MDC/HARD/hardController/MDCHARD050.cmd#none)

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko'}

df_list = []
for year in tqdm(range(2011, 2018 + 1)):
    data = {'bld': 'dbms/MDC/HARD/MDCHARD05001',
    'locale': 'ko_KR',
    'selTp': '1','grdYy': year}
    response = requests.post('https://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', headers = headers, data = data)
    df_list.append(pd.DataFrame(response.json()['block1']))

df_esg_1118 = pd.concat(df_list).drop(columns = 'RN').reset_index(drop = True)
df_esg_1118.sample(10)

100%|██████████| 8/8 [00:01<00:00,  7.56it/s]


,ISU_NM,GRD1,GRD2,GRD3,GRD4,YY
5159,퍼시스,B이하,B,B이하,B이하,2018
5180,한국공항,B이하,C,B이하,B이하,2018
5130,코오롱글로벌,B이하,B+,B이하,B이하,2018
4563,JW중외제약,B이하,B,B+,B이하,2017
685,F&F,B이하,B,B이하,B이하,2012
1788,화승인더스트리,B이하,C,B이하,B+,2013
4103,아남전자,B이하,B,B이하,B이하,2016
2537,한화케미칼,B+,B,A,B+,2014
1760,현대건설,B+,B+,B+,B+,2013
4521,신세계건설,B이하,B+,B이하,B이하,2017


## 전처리

In [8]:
df_esg_1118.columns = ['종목명','ESG종합','E','S','G','평가년도']
df_esg_1118 = df_esg_1118.sort_values(by="평가년도", ascending=False) 
df_esg_1118 = df_esg_1118.reset_index(drop=True)
df_esg_1118

,종목명,ESG종합,E,S,G,평가년도
0,서울도시가스,B이하,B,B이하,B이하,2018
1,티비에이치글로벌,B이하,B,B이하,B이하,2018
2,평화산업,B이하,B,B이하,B+,2018
3,페이퍼코리아,B이하,C,B이하,B이하,2018
4,퍼시스,B이하,B,B이하,B이하,2018
...,...,...,...,...,...,...
5649,성신양회,B이하,B+,B이하,B이하,2011
5650,성보화학,,B,,,2011
5651,성문전자,B이하,C,B이하,B이하,2011
5652,선창산업,B이하,B,B이하,B이하,2011


## 종목코드
- FinanceDataReader로 KRX 전체 종목 가져오기

In [9]:
df_krx = fdr.StockListing("KRX")
df_krx = df_krx[['Name','Code']]
df_krx.columns = ['종목명','종목코드']
df_krx.head()

,종목명,종목코드
0,삼성전자,005930
1,LG에너지솔루션,373220
2,SK하이닉스,000660
3,삼성바이오로직스,207940
4,삼성SDI,006400


In [10]:
df_esg_1118 = df_esg_1118.merge(df_krx, on="종목명", how="left")
df_esg_1118 = df_esg_1118[["종목코드", "종목명", "평가년도", 'ESG종합','E','S','G']]
df_esg_1118

,종목코드,종목명,평가년도,ESG종합,E,S,G
0,NaN,서울도시가스,2018,B이하,B,B이하,B이하
1,NaN,티비에이치글로벌,2018,B이하,B,B이하,B이하
2,090080,평화산업,2018,B이하,B,B이하,B+
3,001020,페이퍼코리아,2018,B이하,C,B이하,B이하
4,016800,퍼시스,2018,B이하,B,B이하,B이하
...,...,...,...,...,...,...,...
5649,004980,성신양회,2011,B이하,B+,B이하,B이하
5650,003080,성보화학,2011,,B,,
5651,014910,성문전자,2011,B이하,C,B이하,B이하
5652,NaN,선창산업,2011,B이하,B,B이하,B이하


## (선택사항) 비상장종목
- 비상장종목의 경우 종목코드가 NaN임.
- 나중에 KOSPI200 데이터와 합칠 때 NaN 값이 자연스럽게 삭제될 것이라 생각하여
- 기업코드의 NaN 값을 그대로 두기로 함!

In [11]:
df_esg_1118[df_esg_1118['종목코드'].isnull()]

,종목코드,종목명,평가년도,ESG종합,E,S,G
0,NaN,서울도시가스,2018,B이하,B,B이하,B이하
1,NaN,티비에이치글로벌,2018,B이하,B,B이하,B이하
17,NaN,태양금속공업,2018,B이하,C,B이하,B+
20,NaN,태경화학,2018,B이하,C,B이하,B이하
23,NaN,포스코,2018,A,A+,A,A
...,...,...,...,...,...,...,...
5630,NaN,삼호,2011,B이하,B,B이하,B이하
5639,NaN,세원셀론텍,2011,B이하,B,B이하,
5643,NaN,세아베스틸,2011,B이하,B,B이하,B이하
5645,NaN,선진지주,2011,B이하,C,B이하,


In [12]:
df_esg_1118_dn = df_esg_1118.dropna(axis=0)
df_esg_1118_dn.info()
df_esg_1118_dn

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4265 entries, 2 to 5653
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목코드    4265 non-null   object
 1   종목명     4265 non-null   object
 2   평가년도    4265 non-null   object
 3   ESG종합   4265 non-null   object
 4   E       4265 non-null   object
 5   S       4265 non-null   object
 6   G       4265 non-null   object
dtypes: object(7)
memory usage: 266.6+ KB


,종목코드,종목명,평가년도,ESG종합,E,S,G
2,090080,평화산업,2018,B이하,B,B이하,B+
3,001020,페이퍼코리아,2018,B이하,C,B이하,B이하
4,016800,퍼시스,2018,B이하,B,B이하,B이하
5,010820,퍼스텍,2018,B이하,B,B이하,B이하
6,028670,팬오션,2018,B+,B+,B이하,B이하
...,...,...,...,...,...,...,...
5648,011300,성안,2011,B이하,C,B이하,
5649,004980,성신양회,2011,B이하,B+,B이하,B이하
5650,003080,성보화학,2011,,B,,
5651,014910,성문전자,2011,B이하,C,B이하,B이하


## (선택) 영속화

In [13]:
m.DfPrst(df_esg_1118, fp_esg_1118)

['../data/esg_1118.parquet']


# ```KCGS_ESGRating(2021).csv```
  - 개요
    - 2021년 ESG등급
  - Data Collection Method
    - Web Scraping
  - Data Source
    - [KCGS, ESG기준원](http://www.cgs.or.kr/business/esg_tab04.jsp?pg={}&pp=10&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents)
    - [1페이지](http://www.cgs.or.kr/business/esg_tab04.jsp?pg=1&pp=10&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents)
    - [2페이지](http://www.cgs.or.kr/business/esg_tab04.jsp?pg=2&pp=10&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents)
  - Reference
    - [DACON : 이제 ESG에 투자하세요! : 기업 ESG 등급을 이용한 국내 ETF ESG 등급 현황](https://dacon.io/competitions/official/235914/codeshare/5669)
  - 비고
    - 데이터 갱신으로 변경됨.

In [14]:
#각 테마 페이지별 테마명&주식종목명 가져오기
def page_list(page):
    ESGLIST_URL = "http://www.cgs.or.kr/business/esg_tab04.jsp?pg={}&pp=10&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents".format(page) #주소설정
    response = urllib.request.urlopen(ESGLIST_URL)
    ESGLIST_HTML = response.read()

    soup = bs(ESGLIST_HTML)
    STK_ESG_GRADE=pd.DataFrame()
    # 100 페이지까지는 한 페이지에 총 10개 기업 존재
    if page <= 100:
        num_list = [0,1,2,3,4,5,6,7,8,9] 
    # 101 페이지에는 총 5개 기업이 존재
    else:
        num_list = [0,1,2,3,4]
    for i in num_list:
        find_stk = soup.find_all('td')[i*9:i*9+9]
        stk_col_list=[]
        for i in [0,1,2,3,4,5,6,7,8]:
            stk_col_list.append(find_stk[i].text)
            clean_stk_col_list=pd.DataFrame(stk_col_list).transpose()
        STK_ESG_GRADE = pd.concat([STK_ESG_GRADE,clean_stk_col_list])
    return STK_ESG_GRADE

In [15]:
df_esg_21 = pd.DataFrame()
for i in tqdm(list(range(1,102))):
    df_temp = page_list(i)
    df_esg_21 = pd.concat([df_esg_21, df_temp])

df_esg_21.head()

100%|██████████| 101/101 [00:08<00:00, 11.40it/s]


,0,1,2,3,4,5,6,7,8
0,1033,AJ네트웍스,095570,B+,C,B+,B+,2022,
0,1032,AK홀딩스,006840,B+,B,A,B,2022,
0,1031,AP시스템,265520,D,D,C,C,2022,
0,1030,BGF,027410,B+,A,A+,B,2022,
0,1029,BGF리테일,282330,A,A,A+,A,2022,


## 전처리

In [16]:
df_esg_21.columns = ['NO', '종목명','종목코드','ESG종합','E','S','G','평가년도','비고']
df_esg_21.reset_index(inplace=True)
df_esg_21 = df_esg_21[['종목코드','종목명','평가년도','ESG종합','E','S','G']]
df_esg_21.head()

,종목코드,종목명,평가년도,ESG종합,E,S,G
0,095570,AJ네트웍스,2022,B+,C,B+,B+
1,006840,AK홀딩스,2022,B+,B,A,B
2,265520,AP시스템,2022,D,D,C,C
3,027410,BGF,2022,B+,A,A+,B
4,282330,BGF리테일,2022,A,A,A+,A


## (선택) 영속화

In [17]:
m.DfPrst(df_esg_21, fp_esg_21)

['../data/esg_21.parquet']


# [미작업] ```KCGS_ESGRating(2020~2023).csv```
  - 개요
    - 2020~2023년 ESG등급
  - Data Collection Method
    - Web Scraping
  - Data Source
    - [KRX_ESG포털](https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp)

# ```esgRating.csv```

## Data Load

In [18]:
if glob(fp_esg_1118):
    df_esg_1118 = m.DataLoad(fp_esg_1118)
df_esg_1118 = df_esg_1118

if glob(fp_esg_21):
    df_esg_21 = m.DataLoad(fp_esg_21)
df_esg_21 = df_esg_21

Mem. usage decreased to  0.30 Mb (0.0% reduction)


┌▣ df.shape ---- ---- ---- ----
(5654, 7)


┌▣ df.info() ---- ---- ---- ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5654 entries, 0 to 5653
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목코드    4265 non-null   object
 1   종목명     5654 non-null   object
 2   평가년도    5654 non-null   object
 3   ESG종합   5654 non-null   object
 4   E       5654 non-null   object
 5   S       5654 non-null   object
 6   G       5654 non-null   object
dtypes: object(7)
memory usage: 309.3+ KB
None


┌▣ df.head() ---- ---- ---- ----


,종목코드,종목명,평가년도,ESG종합,E,S,G
0,None,서울도시가스,2018,B이하,B,B이하,B이하
1,None,티비에이치글로벌,2018,B이하,B,B이하,B이하
2,090080,평화산업,2018,B이하,B,B이하,B+
3,001020,페이퍼코리아,2018,B이하,C,B이하,B이하
4,016800,퍼시스,2018,B이하,B,B이하,B이하




┌▣ df.columns.to_list() ---- ---- ---- ----
['종목코드', '종목명', '평가년도', 'ESG종합', 'E', 'S', 'G']
Mem. usage decreased to  0.05 Mb (0.0% reduction)


┌▣ df.shape ---- ---- ---- ----
(1005, 7)


┌▣ df.info() ---- ---- ---- ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목코드    1005 non-null   object
 1   종목명     1005 non-null   object
 2   평가년도    1005 non-null   object
 3   ESG종합   1005 non-null   object
 4   E       1005 non-null   object
 5   S       1005 non-null   object
 6   G       1005 non-null   object
dtypes: object(7)
memory usage: 55.1+ KB
None


┌▣ df.head() ---- ---- ---- ----


,종목코드,종목명,평가년도,ESG종합,E,S,G
0,095570,AJ네트웍스,2022,B+,C,B+,B+
1,006840,AK홀딩스,2022,B+,B,A,B
2,265520,AP시스템,2022,D,D,C,C
3,027410,BGF,2022,B+,A,A+,B
4,282330,BGF리테일,2022,A,A,A+,A




┌▣ df.columns.to_list() ---- ---- ---- ----
['종목코드', '종목명', '평가년도', 'ESG종합', 'E', 'S', 'G']


## 병합

In [19]:
df_esg = pd.concat([df_esg_1118, df_esg_21])
df_esg = df_esg.sort_values(by=['종목코드','평가년도'] , ascending=True)
df_esg

,종목코드,종목명,평가년도,ESG종합,E,S,G
522,000010,신한은행,2022,-,-,-,B+
5095,000020,동화약품,2011,B이하,B+,B이하,B이하
4701,000020,동화약품,2012,B이하,B+,B이하,B이하
3595,000020,동화약품,2013,B이하,B,B이하,B이하
2904,000020,동화약품,2014,B이하,B,B이하,B이하
...,...,...,...,...,...,...,...
725,None,대덕GDS,2018,B+,B,B+,B+
731,None,골든브릿지증권,2018,B이하,B+ 이하,B이하,B이하
733,None,고려개발,2018,B이하,B,B+,B이하
734,None,계룡건설산업,2018,B이하,B,B+,B이하


## 전처리

In [20]:
# 정렬
df_esg = df_esg.sort_values(by=["종목코드", "평가년도"], ascending=[True, True])

# 결측치
col_esg = ["ESG종합", "E", "S", "G"]
for i in col_esg:
    df_esg.loc[df_esg[i] == "-", i] = np.nan
df_esg

,종목코드,종목명,평가년도,ESG종합,E,S,G
522,000010,신한은행,2022,NaN,NaN,NaN,B+
5095,000020,동화약품,2011,B이하,B+,B이하,B이하
4701,000020,동화약품,2012,B이하,B+,B이하,B이하
3595,000020,동화약품,2013,B이하,B,B이하,B이하
2904,000020,동화약품,2014,B이하,B,B이하,B이하
...,...,...,...,...,...,...,...
725,None,대덕GDS,2018,B+,B,B+,B+
731,None,골든브릿지증권,2018,B이하,B+ 이하,B이하,B이하
733,None,고려개발,2018,B이하,B,B+,B이하
734,None,계룡건설산업,2018,B이하,B,B+,B이하


### (선택) 등급에 '이하' 통합
- (예시) 'B+ 이하'는 'B+'로 변경하여 통합함

In [21]:
def esgRating(x):
    try:
        dict_enc = {
            "S": "S",
            "A+": "A+",
            "A": "A",
            "B+": "B+",
            "B+ 이하": "B+",
            "B": "B",
            "B이하": "B",
            "C": "C",
            "D": "D",
            "-": np.nan,
            "": np.nan,
            "NaN": np.nan,
            "None": np.nan
        }
        return dict_enc[x]
    except:
        return np.nan


for i in col_esg:
    df_esg[i] = df_esg[i].fillna(np.nan).apply(esgRating)
    
df_esg

,종목코드,종목명,평가년도,ESG종합,E,S,G
522,000010,신한은행,2022,NaN,NaN,NaN,B+
5095,000020,동화약품,2011,B,B+,B,B
4701,000020,동화약품,2012,B,B+,B,B
3595,000020,동화약품,2013,B,B,B,B
2904,000020,동화약품,2014,B,B,B,B
...,...,...,...,...,...,...,...
725,None,대덕GDS,2018,B+,B,B+,B+
731,None,골든브릿지증권,2018,B,B+,B,B
733,None,고려개발,2018,B,B,B+,B
734,None,계룡건설산업,2018,B,B,B+,B


### Derived Column - Encoding

In [22]:
def esgRating_enc(x):
    try:
        dict_enc = {
            "S": "6",
            "A+": "5",
            "A": "4",
            "B+": "3",
            "B+ 이하": "3",
            "B": "2",
            "B이하": "2",
            "C": "1",
            "D": "0",
            "-": np.nan,
            "": np.nan,
            "NaN": np.nan,
            "None": np.nan
        }
        return dict_enc[x]
    except:
        return np.nan


col_esg = ["ESG종합", "E", "S", "G"]
for i in col_esg:
    df_esg[f"{i}_enc"] = df_esg[i].fillna(np.nan).apply(esgRating_enc)
    
df_esg

,종목코드,종목명,평가년도,ESG종합,E,S,G,ESG종합_enc,E_enc,S_enc,G_enc
522,000010,신한은행,2022,NaN,NaN,NaN,B+,NaN,NaN,NaN,3
5095,000020,동화약품,2011,B,B+,B,B,2,3,2,2
4701,000020,동화약품,2012,B,B+,B,B,2,3,2,2
3595,000020,동화약품,2013,B,B,B,B,2,2,2,2
2904,000020,동화약품,2014,B,B,B,B,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...
725,None,대덕GDS,2018,B+,B,B+,B+,3,2,3,3
731,None,골든브릿지증권,2018,B,B+,B,B,2,3,2,2
733,None,고려개발,2018,B,B,B+,B,2,2,3,2
734,None,계룡건설산업,2018,B,B,B+,B,2,2,3,2


## 영속화

In [23]:
m.DfPrst(df_esg, fp_esg)

['../data/esgRating.parquet']


### (선택) 필요없는 파일 제거

In [47]:
# if os.path.isfile(fp_esg_1118):
#     os.remove(fp_esg_1118)

# if os.path.isfile(fp_esg_21):
#     os.remove(fp_esg_21)